이용 건수는 중복값을 처리해서 계산하면 안된다.
-> 해당월의 총 이용건수를 봐야하기 때문에.

이용인원만 중복값을 처리한 후 계산한다. 


In [1]:
#### 1. 엑셀 파일 원본 데이터 불러오기 이름:df

import openpyxl
import pandas as pd
import numpy as np

df_og = pd.read_excel("C:\workshop\data\현대자동차서비스2022_1월데이터.xlsx", header = 1)

df = df_og.loc[:,['내담자명', '전화번호', '상담신청경로', '상담주제', '성별', '연령','직군', '직위','중심조직', '효과성 검사 위험도 수준', 
   '참석동반자','상담방법', '상담목표', '상담내용', '상담회차', '상담일지등록일', '기본상담료']]

#nan값 있는 0번째 인덱스 행 지우기 
df = df.drop(index = 0, axis = 0)

#### 2. 해당 날짜 데이터만 남기기(1월달 남기기)

#상담일지 등록일을 기준으로 월보고를 작성한다. (확인필요)
#먼저 날짜 컬럼을 스트링으로 변경한다. 
df['상담일지등록일']= df['상담일지등록일'].astype('str')
#0000-00-00형식의 날짜로 변환한다. 
df['상담일지등록일'] = pd.to_datetime(df['상담일지등록일'],format='%Y-%m-%d')
#해당 날짜에 해당하는 값들 추출한다. 
df = df [df['상담일지등록일'].between('2022-01-01', '2022-01-31')]

#참석 동반자 결측값 직원으로 채우기
df['참석동반자'] = df['참석동반자'].replace(to_replace = np.nan, value = '직원')

## 이용인원구하기 위한 데이터프레임 
#마지막만 남기고 중복값 지우기 (동명이인일 경우를 생각해서 전화번호를 고유값으로 정한다.)
#마지막 데이터를 남기는 이유는 : 해당 내담자가 진행한 마지막 회기(최신상태)만 남기기 위해서
df_people = df.drop_duplicates(['전화번호'], keep='last')

#target_month = 1
# df = df[df['상담일지등록일'].dt.month==target_month] 
# 또는 df1 = df1.query('상담일지등록일.dt.month == @target_month')

# 인덱스 리셋하기
df = df.reset_index(drop = True) # 상담 이용건수 구하는 dataframe
df_people = df_people.reset_index(drop = True) #이용인원 구하는 dataframe

# 운영개요

# 1. 이용실적

In [2]:
print('이용인원에 따른 상담방법',df_people['상담방법'].value_counts())
print('')
print('이용건수에 따른 상담방법',df['상담방법'].value_counts())

이용인원에 따른 상담방법 대면    9
Name: 상담방법, dtype: int64

이용건수에 따른 상담방법 대면    18
Name: 상담방법, dtype: int64


In [3]:
#### 1. 이용실적 표 구하기 
print("2페이지")
# 대면상담 중 가족 이용인원
df_inperson_family_people = df_people[(df_people['상담방법'] == '대면') & (df_people["참석동반자"].str.startswith("가족"))]
df_inperson_family_people = df_inperson_family_people.reset_index()
df_inperson_family_people_num = len(df_inperson_family_people)
print("대면상담 중 가족 이용인원 : ", df_inperson_family_people_num)

# 대면상담 중 가족 이용건수 
df_inperson_family_counts = df[(df['상담방법'] == '대면') & (df["참석동반자"].str.startswith("가족"))]
df_inperson_family_counts = df_inperson_family_counts.reset_index()
df_inperson_family_counts_num = len(df_inperson_family_counts)
print("대면상담 중 가족 이용건수 : ", df_inperson_family_counts_num )

# 대면상담 중 직원 이용인원
df_inperson_employee_people = df_people[(df_people['상담방법'] == '대면') & (df_people["참석동반자"].str.startswith("직원"))]
df_inperson_employee_people = df_inperson_employee_people.reset_index()
df_inperson_employee_people_num = len(df_inperson_employee_people)
print("대면상담 중 직원 이용인원 : ", df_inperson_employee_people_num )

# 대면상담 중 직원 이용건수 
df_inperson_employee_counts = df[(df['상담방법'] == '대면') & (df["참석동반자"].str.startswith("직원"))]
df_inperson_employee_counts = df_inperson_employee_counts.reset_index()
df_inperson_employee_counts_num = len(df_inperson_employee_counts)
print("대면상담 중 직원 이용건수 : ", df_inperson_employee_counts_num )

# 전화상담 중 가족 이용인원
df_call_family_people = df_people[(df_people['상담방법'] == '전화') & (df_people["참석동반자"].str.startswith("가족"))]
df_call_family_people = df_call_family_people.reset_index()
df_call_family_people_num = len(df_call_family_people)
print("전화상담 중 가족 이용인원 : ", df_call_family_people_num)


# 전화상담 중 가족 이용건수 
df_call_family_counts = df[(df['상담방법'] == '전화') & (df["참석동반자"].str.startswith("가족"))]
df_call_family_counts = df_call_family_counts.reset_index()
df_call_family_counts_num = len(df_call_family_counts)
print("전화상담 중 가족 이용건수 : ", df_call_family_counts_num)

# 전화상담 중 직원 이용인원
df_call_employee_people = df_people[(df_people['상담방법'] == '전화') & (df_people["참석동반자"].str.startswith("직원"))]
df_call_employee_people = df_call_employee_people.reset_index()
df_call_employee_people_num = len(df_call_employee_people)
print("전화상담 중 직원 이용인원 : ", df_call_employee_people_num)


# 전화상담 중 직원 이용건수 
df_call_employee_counts = df[(df['상담방법'] == '전화') & (df["참석동반자"].str.startswith("직원"))]
df_call_employee_counts = df_call_employee_counts.reset_index()
df_call_employee_counts_num = len(df_call_employee_counts)
print("전화상담 중 직원 이용건수 : ", df_call_employee_counts_num)


# 화상상담 중 가족 이용인원
df_zoom_family_people = df_people[(df_people['상담방법'] == '화상') & (df_people["참석동반자"].str.startswith("가족"))]
df_zoom_family_people = df_zoom_family_people.reset_index()
df_zoom_family_people_num = len(df_zoom_family_people)
print("화상상담 중 가족 이용인원 : ", df_zoom_family_people_num)

# 화상상담 중 가족 이용건수 
df_zoom_family_counts = df[(df['상담방법'] == '화상') & (df["참석동반자"].str.startswith("가족"))]
df_zoom_family_counts = df_zoom_family_counts.reset_index()
df_zoom_family_counts_num = len(df_zoom_family_counts)
print("화상상담 중 가족 이용건수 : ", df_zoom_family_counts_num)

# 화상상담 중 직원 이용인원
df_zoom_employee_people = df_people[(df_people['상담방법'] == '화상') & (df_people["참석동반자"].str.startswith("직원"))]
df_zoom_employee_people = df_zoom_employee_people.reset_index()
df_zoom_employee_people_num = len(df_zoom_employee_people)
print("화상상담 중 직원 이용인원 : ", df_zoom_employee_people_num)

# 화상상담 중 직원 이용건수 
df_zoom_employee_counts = df[(df['상담방법'] == '화상') & (df["참석동반자"].str.startswith("직원"))]
df_zoom_employee_counts = df_zoom_employee_counts.reset_index()
df_zoom_employee_counts_num = len(df_zoom_employee_counts)
print("화상상담 중 직원 이용건수 : ", df_zoom_employee_counts_num)

# 이메일 중 가족 이용인원
df_email_family_people = df_people[(df_people['상담방법'] == '이메일') & (df_people["참석동반자"].str.startswith("가족"))]
df_email_family_people = df_email_family_people.reset_index()
df_email_family_people_num = len(df_email_family_people)
print("이메일 중 가족 이용인원 : ", df_email_family_people_num)

# 이메일 중 가족 이용건수 
df_email_family_counts = df[(df['상담방법'] == '이메일') & (df["참석동반자"].str.startswith("가족"))]
df_email_family_counts = df_email_family_counts.reset_index()
df_email_family_counts_num = len(df_email_family_counts)
print("이메일 중 가족 이용건수  : ", df_email_family_counts_num)

# 이메일 중 직원 이용인원
df_email_employee_people = df_people[(df_people['상담방법'] == '이메일') & (df_people["참석동반자"].str.startswith("직원"))]
df_email_employee_people = df_email_employee_people.reset_index()
df_email_employee_people_num = len(df_email_employee_people)
print("이메일 중 직원 이용인원 : ", df_email_employee_people_num)


# 이메일 중 직원 이용건수 
df_email_employee_counts = df[(df['상담방법'] == '이메일') & (df["참석동반자"].str.startswith("직원"))]
df_email_employee_counts = df_email_employee_counts.reset_index()
df_email_employee_counts_num = len(df_email_employee_counts)
print("이메일 중 직원 이용건수 : ", df_email_employee_counts_num)

# 톡상담 중 가족 이용인원
df_talk_family_people = df_people[(df_people['상담방법'] == '톡상담') & (df_people["참석동반자"].str.startswith("가족"))]
df_talk_family_people = df_email_family_people.reset_index()
df_talk_family_people_num = len(df_talk_family_people)
print("톡상담 중 가족 이용인원 : ", df_talk_family_people_num)

# 톡상담 중 가족 이용건수 
df_talk_zoom_family_counts = df[(df['상담방법'] == '톡상담') & (df["참석동반자"].str.startswith("가족"))]
df_talk_zoom_family_counts = df_talk_zoom_family_counts.reset_index()
df_talk_zoom_family_counts_num = len(df_talk_zoom_family_counts)
print("톡상담 중 가족 이용건수 : ", df_talk_zoom_family_counts_num)

# 톡상담 중 직원 이용인원
df_talk_zoom_employee_people = df_people[(df_people['상담방법'] == '톡상담') & (df_people["참석동반자"].str.startswith("직원"))]
df_talk_zoom_employee_people = df_talk_zoom_employee_people.reset_index()
df_talk_zoom_employee_people_num = len(df_talk_zoom_employee_people)
print("톡상담 중 직원 이용인원 : ", df_talk_zoom_employee_people_num)

# 톡상담 중 직원 이용건수 
df_talk_employee_counts = df[(df['상담방법'] == '톡상담') & (df["참석동반자"].str.startswith("직원"))]
df_talk_employee_counts = df_talk_employee_counts.reset_index()
df_talk_employee_counts_num = len(df_talk_employee_counts)
print("톡상담 중 직원 이용건수 : ", df_talk_employee_counts_num)


# 야간상담 중 가족 이용인원
df_night_family_people = df_people[(df_people['상담방법'] == '야간상담') & (df_people["참석동반자"].str.startswith("가족"))]
df_night_family_people = df_night_family_people.reset_index()
df_night_family_people_num = len(df_night_family_people)
print("야간상담 중 가족 이용인원 : ", df_night_family_people_num)

# 야간상담 중 가족 이용건수 
df_night_zoom_family_counts = df[(df['상담방법'] == '야간상담') & (df["참석동반자"].str.startswith("가족"))]
df_night_zoom_family_counts = df_night_zoom_family_counts.reset_index()
df_night_zoom_family_counts_num = len(df_night_zoom_family_counts)
print("야간상담 중 가족 이용건수 : ", df_night_zoom_family_counts_num)

# 야간상담 중 직원 이용인원
df_night_zoom_employee_people = df_people[(df_people['상담방법'] == '야간상담') & (df_people["참석동반자"].str.startswith("직원"))]
df_night_zoom_employee_people = df_night_zoom_employee_people.reset_index()
df_night_zoom_employee_people_num = len(df_night_zoom_employee_people)
print("야간상담 중 직원 이용인원 : ", df_night_zoom_employee_people_num)

# 야간상담 중 직원 이용건수 
df_night_employee_counts = df[(df['상담방법'] == '야간상담') & (df["참석동반자"].str.startswith("직원"))]
df_night_employee_counts = df_night_employee_counts.reset_index()
df_night_employee_counts_num = len(df_night_employee_counts)
print("야간상담 중 직원 이용건수 : ", df_night_employee_counts_num)

# 상주상담 중 가족 이용인원
df_stay_family_people = df_people[(df_people['상담방법'] == '상주상담') & (df_people["참석동반자"].str.startswith("가족"))]
df_stay_family_people = df_stay_family_people.reset_index()
df_stay_family_people_num = len(df_stay_family_people)
print("상주상담 중 가족 이용인원 : ", df_stay_family_people_num)

# 상주상담 중 가족 이용건수 
df_stay_zoom_family_counts = df[(df['상담방법'] == '상주상담') & (df["참석동반자"].str.startswith("가족"))]
df_stay_zoom_family_counts = df_stay_zoom_family_counts.reset_index()
df_stay_zoom_family_counts_num = len(df_stay_zoom_family_counts)
print("상주상담 중 가족 이용건수 : ", df_stay_zoom_family_counts_num)


# 상주상담 중 직원 이용인원
df_stay_zoom_employee_people = df_people[(df_people['상담방법'] == '상주상담') & (df_people["참석동반자"].str.startswith("직원"))]
df_stay_zoom_employee_people = df_stay_zoom_employee_people.reset_index()
df_stay_zoom_employee_people_num = len(df_stay_zoom_employee_people)
print("상주상담 중 직원 이용인원 : ", df_stay_zoom_employee_people_num)

# 상주상담 중 직원 이용건수 
df_stay_employee_counts = df[(df['상담방법'] == '상주상담') & (df["참석동반자"].str.startswith("직원"))]
df_stay_employee_counts = df_stay_employee_counts.reset_index()
df_stay_employee_counts_num = len(df_stay_employee_counts)
print("상주상담 중 직원 이용건수 : ", df_stay_employee_counts_num)

# 집단상담 중 가족 이용인원
df_group_family_people = df_people[(df_people['상담방법'] == '집단상담') & (df_people["참석동반자"].str.startswith("가족"))]
df_group_family_people = df_group_family_people.reset_index()
df_group_family_people_num = len(df_group_family_people)
print("집단상담 중 가족 이용인원 : ", df_group_family_people_num)

# 집단상담 중 가족 이용건수 
df_group_family_counts = df[(df['상담방법'] == '집단상담') & (df["참석동반자"].str.startswith("가족"))]
df_group_family_counts = df_group_family_counts.reset_index()
df_group_family_counts_num = len(df_group_family_counts)
print("집단상담 중 가족 이용건수 : ", df_group_family_counts_num)

# 집단상담 중 직원 이용인원
df_group_employee_people = df_people[(df_people['상담방법'] == '집단상담') & (df_people["참석동반자"].str.startswith("직원"))]
df_group_employee_people = df_group_employee_people.reset_index()
df_group_employee_people_num = len(df_group_employee_people)
print("집단상담 중 직원 이용인원 : ", df_group_employee_people_num)

# 집단상담 중 직원 이용건수 
df_group_employee_counts = df[(df['상담방법'] == '집단상담') & (df["참석동반자"].str.startswith("직원"))]
df_group_employee_counts = df_group_employee_counts.reset_index()
df_group_employee_counts_num = len(df_group_employee_counts)
print("집단상담 중 직원 이용건수 : ", df_group_employee_counts_num)

# 노쇼 중 가족 이용인원
df_noshow_family_people = df_people[(df_people['상담방법'] == '집단상담') & (df_people["참석동반자"].str.startswith("가족"))]
df_noshow_family_people = df_noshow_family_people.reset_index()
df_noshow_family_people_num = len(df_noshow_family_people)
print("집단상담 중 가족 이용인원 : ", df_noshow_family_people_num)

# 노쇼 중 가족 이용건수 
df_noshow_zoom_family_counts = df[(df['상담방법'] == '노쇼') & (df["참석동반자"].str.startswith("가족"))]
df_noshow_zoom_family_counts = df_noshow_zoom_family_counts.reset_index()
df_noshow_zoom_family_counts_num = len(df_noshow_zoom_family_counts)
print("노쇼 중 가족 이용건수 : ", df_noshow_zoom_family_counts_num)

# 노쇼 중 직원 이용인원
df_noshow_zoom_employee_people = df_people[(df_people['상담방법'] == '노쇼') & (df_people["참석동반자"].str.startswith("직원"))]
df_noshow_zoom_employee_people = df_noshow_zoom_employee_people.reset_index()
df_noshow_zoom_employee_people_num = len(df_noshow_zoom_employee_people)
print("노쇼 중 직원 이용인원 : ", df_noshow_zoom_employee_people_num)

# 노쇼 중 직원 이용건수 
df_noshow_employee_counts = df[(df['상담방법'] == '노쇼') & (df["참석동반자"].str.startswith("직원"))]
df_noshow_employee_counts = df_noshow_employee_counts.reset_index()
df_noshow_employee_counts_num = len(df_noshow_employee_counts)
print("노쇼 중 직원 이용건수 : ", df_noshow_employee_counts_num)

2페이지
대면상담 중 가족 이용인원 :  4
대면상담 중 가족 이용건수 :  10
대면상담 중 직원 이용인원 :  5
대면상담 중 직원 이용건수 :  8
전화상담 중 가족 이용인원 :  0
전화상담 중 가족 이용건수 :  0
전화상담 중 직원 이용인원 :  0
전화상담 중 직원 이용건수 :  0
화상상담 중 가족 이용인원 :  0
화상상담 중 가족 이용건수 :  0
화상상담 중 직원 이용인원 :  0
화상상담 중 직원 이용건수 :  0
이메일 중 가족 이용인원 :  0
이메일 중 가족 이용건수  :  0
이메일 중 직원 이용인원 :  0
이메일 중 직원 이용건수 :  0
톡상담 중 가족 이용인원 :  0
톡상담 중 가족 이용건수 :  0
톡상담 중 직원 이용인원 :  0
톡상담 중 직원 이용건수 :  0
야간상담 중 가족 이용인원 :  0
야간상담 중 가족 이용건수 :  0
야간상담 중 직원 이용인원 :  0
야간상담 중 직원 이용건수 :  0
상주상담 중 가족 이용인원 :  0
상주상담 중 가족 이용건수 :  0
상주상담 중 직원 이용인원 :  0
상주상담 중 직원 이용건수 :  0
집단상담 중 가족 이용인원 :  0
집단상담 중 가족 이용건수 :  0
집단상담 중 직원 이용인원 :  0
집단상담 중 직원 이용건수 :  0
집단상담 중 가족 이용인원 :  0
노쇼 중 가족 이용건수 :  0
노쇼 중 직원 이용인원 :  0
노쇼 중 직원 이용건수 :  0


# 2. 월별이용현황

In [4]:
print('월별이용현황, 이용인원')
print(df_people['참석동반자'].value_counts())
print('')
print('월별이용현황, 이용건수')
print(df['참석동반자'].value_counts())
print('')

월별이용현황, 이용인원
직원          5
가족 - 배우자    3
가족 - 자녀     1
Name: 참석동반자, dtype: int64

월별이용현황, 이용건수
직원          8
가족 - 배우자    5
가족 - 자녀     5
Name: 참석동반자, dtype: int64



# 3. 인적사항

In [5]:
print("인적사항_성별 구분, 이용인원")
print(df_people['성별'].value_counts())
print('')
print("인적사항_성별 구분, 이용건수")
print(df['성별'].value_counts())
print('')
print("인적사항_연령별 구분, 이용인원")
print(df_people['연령'].value_counts())
print('')
print("인적사항_연령별 구분, 이용건수")
print(df['연령'].value_counts())
print('')
print("인적사항_직군별 구분, 이용인원")
print(df_people['직군'].value_counts())
print('')
print("인적사항_직군별 구분, 이용건수")
print(df['직군'].value_counts())
print('')
print("인적사항_직위별 구분, 이용인원")
print(df_people['직위'].value_counts())
print('')
print("인적사항_직위별 구분, 이용건수")
print(df['직위'].value_counts())
print('')
print("인적사항_중심조직별 구분, 이용인원")
print(df_people['중심조직'].value_counts())
print('')
print("인적사항_중심조직별 구분, 이용건수")
print(df['중심조직'].value_counts())
print('')

인적사항_성별 구분, 이용인원
남    8
여    1
Name: 성별, dtype: int64

인적사항_성별 구분, 이용건수
남    17
여     1
Name: 성별, dtype: int64

인적사항_연령별 구분, 이용인원
40대    3
50대    3
30대    2
20대    1
Name: 연령, dtype: int64

인적사항_연령별 구분, 이용건수
50대    7
30대    5
40대    5
20대    1
Name: 연령, dtype: int64

인적사항_직군별 구분, 이용인원
정비    5
일반    4
Name: 직군, dtype: int64

인적사항_직군별 구분, 이용건수
정비    12
일반     6
Name: 직군, dtype: int64

인적사항_직위별 구분, 이용인원
매니저       2
정비수석기사    2
임원급       1
정비선임기사    1
정비4급기사    1
책임매니저     1
정비5급기사    1
Name: 직위, dtype: int64

인적사항_직위별 구분, 이용건수
정비4급기사    4
정비수석기사    4
임원급       3
정비선임기사    3
매니저       2
책임매니저     1
정비5급기사    1
Name: 직위, dtype: int64

인적사항_중심조직별 구분, 이용인원
본사           2
서부산하이테크센터    2
동부하이테크센터     1
창원하이테크센터     1
울산하이테크센터     1
전주하이테크센터     1
대전하이테크센터     1
Name: 중심조직, dtype: int64

인적사항_중심조직별 구분, 이용건수
서부산하이테크센터    5
본사           4
울산하이테크센터     3
동부하이테크센터     2
창원하이테크센터     2
전주하이테크센터     1
대전하이테크센터     1
Name: 중심조직, dtype: int64



# 4. 이용자 현황

In [6]:
print("4. 이용자 현황_월별 신규인원")
df신규인원 = df['상담회차'] == '1회차'
## flase를 세면됨 
print(df신규인원.value_counts())
print("")

print("4. 이용자 현황_상담신청 경로")
print(df_people['상담신청경로'].value_counts())
print("notice : 기타는 의뢰입니다")
print("")

print("4. 이용자 현황_내담자 마음건강 상태")
print(df_people['효과성 검사 위험도 수준'].value_counts())

4. 이용자 현황_월별 신규인원
False    9
True     9
Name: 상담회차, dtype: int64

4. 이용자 현황_상담신청 경로
캠페인            4
홍보레터           3
기타CUSCUS099    2
Name: 상담신청경로, dtype: int64
notice : 기타는 의뢰입니다

4. 이용자 현황_내담자 마음건강 상태
무응답    5
예방군    2
건강군    1
안정군    1
Name: 효과성 검사 위험도 수준, dtype: int64


# 분석결과

# 1. EAP 상담 진행 내용

In [7]:
print("1) 회기별 분류")
print(df_people['상담회차'].value_counts())
print("")

print("2) 상담분야별분류_이용인원")
print(df_people['상담주제'].value_counts())
print("")

print("2) 상담분야별분류_이용건수")
print(df['상담주제'].value_counts())
print("")

print("[개인/조직 상담 total]")
print("False : 개인, True :  조직")
df_개인 = df_people['상담주제'].str.startswith("Work")
print(df_개인.value_counts())

df_people['참석동반자'] == '직원'

1) 회기별 분류
1회차    4
2회차    2
3회차    2
4회차    1
Name: 상담회차, dtype: int64

2) 상담분야별분류_이용인원
Life-부부/가족      4
Work-직무적응/적성    2
Life-관계(갈등)     1
Life-자녀양육       1
Life-건강         1
Name: 상담주제, dtype: int64

2) 상담분야별분류_이용건수
Life-부부/가족      10
Work-직무적응/적성     5
Life-관계(갈등)      1
Life-자녀양육        1
Life-건강          1
Name: 상담주제, dtype: int64

[개인/조직 상담 total]
False : 개인, True :  조직
False    7
True     2
Name: 상담주제, dtype: int64


0     True
1    False
2     True
3    False
4    False
5     True
6    False
7     True
8     True
Name: 참석동반자, dtype: bool

In [8]:
df.to_excel("C:\workshop\상담주제확인용.xlsx")

In [9]:
#데이터 현황 파악하기 
#people = []
#count_p = 0
#count_c = 0

#for i in df['전화번호']:
#    count_c += 1
#    if i in people:
#        pass
#    else:
#        people.append(i)
#        count_p += 1

#print("내담자 수 :", count_p)
#print("상담건수 :", count_c)
#print("내담자 이름 :", people)

In [10]:
#데이터 현황 파악하기 
#ppl = []
#counts = 0
#counsel = 0

#for p in df['내담자명']:
#    counsel += 1
#    if p in ppl:
#        pass
#    else:
#        ppl.append(p)
#         ppl.remove(nan)
#        counts += 1
    
#print('내담자 수 :',counts)
#print('상담건수 :', counsel)
#print('내담자 이름 :', ppl)

In [11]:
#1월달 데이터만 남기기
#df2['상담일지등록일'] = pd.to_datetime(df2['상담일지등록일'],format='%Y-%m-%d')
#df2['상담일지등록일']
#df1['상담일지등록일'] = pd.to_datetime(df1['상담일지등록일'],format='%Y-%m-%d')
#target_month = 1 # 1월
#df.query('상담일지등록일.dt.month == @target_month') ## 또는 df[df['Date'].dt.month==target_month]